# Bulk processing

Take all the jump files in a data lake and process them as a group.  Produce the mean results for scores, max speed, curves, and the 5-second partial results.

Conforming to Lucyfer's default configuration, the data lake starts at the `./data` directory and includes any and all files below it.

---
## Prepare the environment

In [101]:
from collections import namedtuple
from copy import deepcopy

from ssscoring import ALTITUDE_SKYDIVE_PARACLETE_XP
from ssscoring import BREAKOFF_ALTITUDE
from ssscoring import PERFORMANCE_WINDOW_LENGTH
from ssscoring import convertFlySight2SSScoring
from ssscoring import dropNonSkydiveDataFrom
from ssscoring import getAllSpeedJumpFilesFrom
from ssscoring import getSpeedSkydiveFrom
from ssscoring import isValidJump
from ssscoring import jumpAnalysisTable
from ssscoring import validFlySightHeaderIn
from ssscoring.notebook import processJump

import csv
import os
import os.path as path

import bokeh.plotting as bp
import ipywidgets as widgets
import pandas as pd

In [102]:
DATA_LAKE_ROOT = './data' # Lucyfer default
FLYSIGHT_HEADER = set([ 'time', 'lat', 'lon', 'hMSL', 'velN', 'velE', 'velD', 'hAcc', 'vAcc', 'sAcc', 'heading', 'cAcc', 'gpsFix', 'numSV', ])
IGNORE_LIST = [ '.ipynb_checkpoints', ]
MIN_JUMP_FILE_SIZE = 1024*512
LAST_TIME_TRANCHE = 25.0
SPEED_COLORS = colors = ('blue', 'limegreen', 'tomato', 'turquoise', 'deepskyblue', 'forestgreen', 'coral', 'darkcyan',)

In [103]:
bp.output_notebook()

Loading BokehJS ...

## Get a list of all FlySight files in the data lake

This also discards all files that don't reflect a valid jump:

- Detect the files by size and discard any file smaller than `MIN_JUMP_FILE_SIZE`
- Detect that each file has the FlySight header on the first line

In [104]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

---
## Process all files

In [105]:
resultsIndex = pd.Series((tag for tag in jumpFiles))

In [106]:
jumpResults = dict()
for jumpFile in jumpFiles:
    jumpResult = processJump(
        convertFlySight2SSScoring(
            pd.read_csv(jumpFile, skiprows = (1, 1)),
            altitudeDZMeters = ALTITUDE_SKYDIVE_PARACLETE_XP))
    tag = jumpFile.replace('CSV', '').replace('.', '').replace('/data', '').replace('/', ' ').strip()
    if 'valid' in jumpResult.result:
        jumpResults[tag] = jumpResult

---
## Results

In [107]:
def aggregateResults(jumpResults: dict) -> pd.DataFrame:
    ref = 0
    speeds = pd.DataFrame()
    for jumpResultIndex in jumpResults.keys():
        jumpResult = jumpResults[jumpResultIndex]
        if 'invalid' not in jumpResult.result:
            t = jumpResult.table
            finalTime = t.iloc[-1].time
            t.iloc[-1].time = LAST_TIME_TRANCHE
            t = pd.pivot_table(t, columns = t.time)
            t.drop(['altitude (ft)'], inplace = True)
            d = pd.DataFrame([ jumpResult.score, ], index = [ jumpResultIndex, ], columns = [ 'score', ], dtype = object)
            for column in t.columns:
                d[column] = t[column][0]
            d['finalTime'] = [ finalTime, ]
            d['maxSpeed'] = jumpResult.maxSpeed
    
            if speeds.empty:
                speeds = d.copy()
            else:
                speeds = pd.concat([ speeds, d, ])
    return speeds

In [108]:
aggregate = aggregateResults(jumpResults)
sum = pd.DataFrame({ 'totalSpeed': [ aggregate.score.sum(), ], 'meanSpeed': [ aggregate.score.mean(), ], }, index = [ 'totalSpeed'],)
display(aggregate)
display(sum)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
J01-08-13-13,414.918,178.056,310.824,385.344,406.944,414.540,25.0,418.176
J02-09-25-25,448.848,180.252,307.692,390.204,431.460,450.828,25.0,452.448
J03-10-32-58,452.304,177.984,310.284,398.700,444.420,452.448,25.0,453.888
J04-11-46-19,441.09,179.568,304.560,386.100,428.076,439.884,25.0,442.260
J05-13-02-44,428.202,178.740,314.388,397.296,416.088,420.768,25.0,431.208
J06-14-19-40,433.584,165.744,283.464,377.820,419.004,432.216,25.0,439.056
J07-07-52-08,441.828,179.100,311.868,398.484,440.748,414.432,25.0,446.544
J08-09-07-09,439.686,180.936,304.524,387.288,431.964,440.856,25.0,443.052


,totalSpeed,meanSpeed
totalSpeed,3500.46,437.5575


In [109]:
def initializePlot(jumpTitle):
    return bp.figure(title = jumpTitle,
                     height = 500,
                     width = 900,
                     x_axis_label = 'seconds from exit',
                     y_axis_label = 'km/h',
                     x_range = (0,40),
                     y_range = (0, 550))    

In [111]:
def graphJumpResult(plot,
                    jumpResult,
                    lineColor = 'green',
                    legend = 'speed',
                    showIt = True):
    data = jumpResult.data
    maxSpeed = jumpResult.maxSpeed
    scores = jumpResult.scores
    score = jumpResult.score
    plot.line(data.plotTime, data.vKMh, legend_label = legend, line_width = 2, line_color = lineColor)
    
    if showIt:
        meanData = pd.DataFrame({ 'plotTime': scores.values(), 'score': scores.keys(), })
        meanData = meanData[meanData.plotTime >= 15.0]
        # plot.line(meanData.plotTime, meanData.score, legend_label = 'mean', line_width = 2, line_color = 'red')
        plot.segment(x0 = [ scores[score], ], # scores[score]-3.0, ],
                  y0 = [ 0.0, ], # 0.0, ],
                  x1 = [ scores[score], ], # scores[score]-3.0, ],
                  y1 = [ score, ], #maxSpeed, ],
                  line_width = 5,
                  color = 'green')
        bp.show(plot)

In [112]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in jumpResults.keys():
    jumpResult = jumpResults[resultRef]
    if 'invalid' in jumpResult.result:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if 'invalid' not in jumpResult.result:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), breakoff at %d m (%d ft)</h3>'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        graphJumpResult(initializePlot(resultRef), jumpResult)
        graphJumpResult(allJumpsPlot, jumpResult, lineColor = SPEED_COLORS[mixColor], legend = '%s - %.2f' % (resultRef, jumpResult.score), showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">J01-08-13-13 jump - 🟢 valid - score = 414.92 km/h</span></h1>')

HTML(value='<h3>Max speed = 418; exit at 4191 m (13750 ft), breakoff at 1935 m (6349 ft)</h3>')

HTML(value='<hr><h1><span style="color: #0f0">J02-09-25-25 jump - 🟢 valid - score = 448.85 km/h</span></h1>')

HTML(value='<h3>Max speed = 452; exit at 4217 m (13838 ft), breakoff at 1961 m (6436 ft)</h3>')

HTML(value='<hr><h1><span style="color: #0f0">J03-10-32-58 jump - 🟢 valid - score = 452.30 km/h</span></h1>')

...

HTML(value='<h3>Max speed = 439; exit at 4126 m (13538 ft), breakoff at 1870 m (6136 ft)</h3>')

HTML(value='<hr><h1><span style="color: #0f0">J07-07-52-08 jump - 🟢 valid - score = 441.83 km/h</span></h1>')

HTML(value='<h3>Max speed = 447; exit at 4081 m (13390 ft), breakoff at 1825 m (5988 ft)</h3>')

HTML(value='<hr><h1><span style="color: #0f0">J08-09-07-09 jump - 🟢 valid - score = 439.69 km/h</span></h1>')

HTML(value='<h3>Max speed = 443; exit at 4160 m (13650 ft), breakoff at 1904 m (6249 ft)</h3>')

## All jumps

In [ ]:
bp.show(allJumpsPlot)